<img width=200 src="https://www.smhi.se/polopoly_fs/1.135796.1527766089!/image/LoggaEUCP.png_gen/derivatives/Original_366px/image/LoggaEUCP.png"> <img width=200 src="https://zenodo.org/api/files/00000000-0000-0000-0000-000000000000/is-enes3/logo.png"> <img width=200 src="https://www.dtls.nl/wp-content/uploads/2015/03/NleSc.png"> <img width=200 src="https://www.dkrz.de/++theme++dkrz.theme/images/logo.png"> <img width=200 src="https://jupyter.org/assets/hublogo.svg"> <img width=200 src="https://docs.esmvaltool.org/en/latest/_static/ESMValTool-logo.png">

# Operate on the preprocessed data in the Jupyter environment
Globally, the execution of ESMValTool consists of two main steps:

1. Applying common pre-processor functions to the data
2. Running a custom diagnostic script that uses the pre-processed data

In a notebook environment, an interesting use case is to skip the second step and analyse the pre-processed data interactively. As such, the notebook provides a very convenient environment to interactively work on some code. But at some point, the notebook can become very long and unorganized. Then, it is both convenient and useful to collect the code in an external (python) script. This script can be edited in the Jupyter Lab environment as well. This may in fact be a very natural workflow to create a diagnostic script.

In [ ]:
# The new API is accessible via the esmvalcore.experimental module. These warnings will disappear in the future.
import warnings; warnings.filterwarnings('ignore')
import esmvalcore.experimental as esmvaltool

In [ ]:
# Running serially is easier for debugging
config = esmvaltool.CFG
config['max_parallel_tasks'] = 1
config

# Start with an (almost) empty draft recipe

Copy the following into a file called `./recipe_warming_stripes.yml`

```Yaml
# ESMValTool
# recipe_warming_stripes.yml
---
documentation:
  description: Reproduce warming stripes figure by Ed Hawkins

  authors:
    - kalverla_peter

  references:
    - acknow_project

datasets:
  - {dataset: BCC-ESM1, project: CMIP6, mip: Amon, exp: historical, ensemble: r1i1p1f1, grid: gn, start_year: 1850, end_year: 2014}

preprocessors: null

diagnostics:
  global_warming:
    variables:
      tas:
    scripts: null
```

# Now load this recipe with ESMValTool

In [ ]:
stripes = esmvaltool.get_recipe('./recipe_warming_stripes.yml')
stripes

In [ ]:
stripes.run()

In [ ]:
output = stripes.get_output()
xrds = output['global_warming/tas'][0].load_xarray()
xrds

# Pre-process the data using ESMValTool's standardized preprocessors

Paste the following snippet into your recipe

```Yaml
preprocessors:
  global_anomalies:
    area_statistics:
      operator: mean
    anomalies:
      period: month
      reference:
        start_year: 1981
        start_month: 1
        start_day: 1
        end_year: 2010
        end_month: 12
        end_day: 31
      standardize: false
```

Also, under the variable `tas`, add 

```Yaml
tas:
  preprocessor: global_anomalies
```

Then run the recipe again:

In [ ]:
stripes = esmvaltool.get_recipe('./recipe_warming_stripes.yml')
output = stripes.run()
xrds = output['global_warming/tas'][0].load_xarray()
xrds

# Let's create a nice visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

time = xrds.time
height = np.array([0, 1])
temperature = np.vstack([xrds.tas.values, xrds.tas.values])

figure, axes = plt.subplots()
axes.pcolormesh(time, height, temperature, cmap='coolwarm', shading='auto')

## Move the analysis code to a script

As the analysis code grows, it might be useful to move it to a script. Of course it can be any script, but in this demo we'll focus on working towards something we can contribute back to ESMValTool

Paste the following boilerplate code into a script called `./stripes.py`

```Python
"""Plot timeseries of global temperature anomalies in the 
form of the popular warming stripes figure by Ed Hawkins.
"""

from pathlib import Path
import xarray as xr
from esmvaltool.diag_scripts.shared import run_diagnostic, save_figure


def main(cfg):
    """Compute the time average for each input dataset."""
    # Get info on the preprocessed data.
    input_data = cfg['input_data'].values()

    # Loop over variables/datasets in alphabetical order
    for dataset in input_data:
        # Load the data with xarray
        input_file = dataset['filename']
        name = dataset['alias']
        xrds = xr.open_dataset(input_file)

        # Do awesome stuff

        
if __name__ == '__main__':

    with run_diagnostic() as config:
        main(config)
```

and make sure ESMValTool also runs the script, by adding the following snippet to the recipe (instead of `scripts: null`).

```Yaml
  scripts:
    stripes:
      script: ~/stripes.py
```

This script will not (yet) produce any output, but let's verify that we can actually run it.

In [ ]:
stripes = esmvaltool.get_recipe('./recipe_warming_stripes.yml')
output = stripes.run()
output

# Add code to the diagnostic script

Let's complete the script by adding our code to the diagnostic script. Note that you'll also need to add the additional import statements.

```Python
# These should be added at the top
import numpy as np
import matplotlib.pyplot as plt

# These lines should be added where the 'awesome stuff' happens
time = xrds.time
height = np.array([0, 1])
temperature = np.vstack([xrds.tas.values, xrds.tas.values])

figure, axes = plt.subplots()
axes.pcolormesh(time, height, temperature, cmap='coolwarm', shading='auto')

# We'll add some extra code to make sure the output is saved and tracked
provenance = {
    'caption': f"Warming stripes for dataset {name}",
    'domains': ['global'],
    'authors': ['kalverla_peter'],
    'references': ['acknow_project'],
    'ancestors': [input_file],
}
save_figure(name, provenance, config, figure)
```

And again, try to run the script

In [ ]:
stripes = esmvaltool.get_recipe('./recipe_warming_stripes.yml')
output = stripes.run()
output

In [ ]:
output['global_warming/stripes'][0]

Now you're ready to make your first pull request :-)

Happy coding!